In [1]:
import sys
import os
# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import pandas as pd
import yaml
from datetime import datetime as dt, date
import requests
import logging
from datetime import datetime, timedelta
from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut
from src.databaseService.Parser import Parser_AV
from src.databaseService.CleanData import CleanData
from src.common.AssetDataService import AssetDataService

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData

import sys

logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]
# Usage
logger.info("This will print to the notebook’s output cell")

This will print to the notebook’s output cell


In [2]:
# Define paths
current_dir = os.getcwd()
desired_folder = "secrets"
absolute_path_to_folder = os.path.join(os.path.abspath(os.path.join(current_dir, "..")), "secrets")

# Path to the YAML file
yaml_file_path = os.path.join("../secrets", "alphaVantage.yaml")

# Read and load the YAML file
try:
    with open(yaml_file_path, 'r') as file:  # Open the YAML file for reading
        config = yaml.safe_load(file)  # Load the YAML content
        apiKey = config['alphaVantage_premium']['apiKey']  # Access the required key
except PermissionError:
    print("Permission denied. Please check file permissions.")
except FileNotFoundError:
    print("File not found. Please verify the path.")
except KeyError:
    print("KeyError: Check the structure of the YAML file.")
except yaml.YAMLError as e:
    print("YAML Error:", e)

In [3]:
ticker = "WILC"
ts = TimeSeries(key=apiKey, output_format='pandas')
fd = FundamentalData(key=apiKey, output_format='pandas')

datashareprice, _ = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
loadad_overview, _ = fd.get_company_overview(symbol=ticker)

url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol='+ticker+'&apikey='+apiKey
incStatementData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol='+ticker+'&apikey='+apiKey
cashFlowData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol='+ticker+'&apikey='+apiKey
balanceSheetData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+apiKey
earningsData = requests.get(url).json()

if incStatementData=={} or cashFlowData == {} or balanceSheetData == {} or earningsData == {}:
    raise ImportError(f"Empty Financial Data")

In [4]:
res = Parser_AV(sharepriceData=datashareprice).parse_shareprice()

print(res.tail(5))
print(res.dtypes)

            Date   Open   High      Low  Close  AdjClose   Volume  Dividends  \
6465  2025-07-17  20.26  20.26  19.5545  19.95     19.95  51808.0        0.0   
6466  2025-07-18  20.05  20.05     19.5  19.82     19.82   8497.0        0.0   
6467  2025-07-21  20.65   21.1    20.49  21.05     21.05  87444.0        0.0   
6468  2025-07-22  21.28  21.45    21.07  21.39     21.39  22870.0        0.0   
6469  2025-07-23  21.33   21.4     20.1  20.42     20.42  28934.0        0.0   

      Splits  
6465     1.0  
6466     1.0  
6467     1.0  
6468     1.0  
6469     1.0  
Date         string[python]
Open                Float64
High                Float64
Low                 Float64
Close               Float64
AdjClose            Float64
Volume              Float64
Dividends           Float64
Splits              Float64
dtype: object


In [5]:
loadad_about, loaded_sector = Parser_AV(overview=loadad_overview).parse_overview()

print("About: ", loadad_about)
print("Sector: ", loaded_sector)

About:  {'Symbol': 'WILC', 'AssetType': 'Common Stock', 'Name': 'G Willi-Food International Ltd', 'Description': 'G. Willi-Food International Ltd. develops, imports, exports, markets and distributes food products globally. The company is headquartered in Yavne, Israel.', 'CIK': '1030997', 'Exchange': 'NASDAQ', 'Currency': 'USD', 'Country': 'USA', 'Sector': 'TRADE & SERVICES', 'Industry': 'WHOLESALE-GROCERIES, GENERAL LINE', 'Address': '24 HAMETSUDA ST INDUSTRIAL ZONE, AZUR ISRAEL, IL', 'OfficialSite': 'https://www.willi-food.co.il', 'FiscalYearEnd': 'December', 'LatestQuarter': '2025-03-31', 'MarketCapitalization': '283477000', 'EBITDA': '79990000', 'PERatio': '13.43', 'PEGRatio': '0', 'BookValue': '43.45', 'DividendPerShare': '0', 'DividendYield': '0.0374', 'EPS': '1.52', 'RevenuePerShareTTM': '42.13', 'ProfitMargin': '0.122', 'OperatingMarginTTM': '0.138', 'ReturnOnAssetsTTM': '0.0704', 'ReturnOnEquityTTM': '0.122', 'RevenueTTM': '584602000', 'GrossProfitTTM': '171311000', 'DilutedEP

In [6]:
financials_annually, financials_quarterly = Parser_AV(
    incStatementData = incStatementData, 
    cashFlowData = cashFlowData, 
    balanceSheetData = balanceSheetData, 
    earningsData = earningsData
).parse_financials()

In [7]:
print(financials_annually.tail(3))
print(financials_annually.dtypes)

   fiscalDateEnding  reportedEPS  grossProfit  totalRevenue        ebit  \
17       2022-12-31       0.8949  143097000.0   498325000.0  38588000.0   
18       2023-12-31       0.6202  120567000.0   543262000.0  20350000.0   
19       2024-12-31        1.371  161334000.0   575795000.0  54874000.0   

        ebitda  totalAssets  totalCurrentLiabilities  totalShareholderEquity  \
17  45096000.0  608432000.0               42628000.0             559444000.0   
18  24896000.0  597033000.0               38181000.0             552235000.0   
19  62176000.0  686380000.0               59929000.0             612940000.0   

    operatingCashflow  
17         14069000.0  
18         33708000.0  
19         42973000.0  
fiscalDateEnding           string[python]
reportedEPS                       Float64
grossProfit                       Float64
totalRevenue                      Float64
ebit                              Float64
ebitda                            Float64
totalAssets                   

In [8]:
print(financials_quarterly.tail(3))
print(financials_quarterly.dtypes)

   fiscalDateEnding reportedDate  reportedEPS  estimatedEPS  surprise  \
78       2024-09-30   2024-12-19        0.403          <NA>       0.0   
79       2024-12-31   2025-03-19       0.4704          <NA>       0.0   
80       2025-03-31   2025-05-19       0.3859          <NA>       0.0   

    surprisePercentage  reportTime  grossProfit  totalRevenue        ebit  \
78                <NA>  pre-market   41962000.0   152799000.0  26993000.0   
79                <NA>  pre-market   38797000.0   140302000.0  -8541000.0   
80                <NA>  pre-market   44628000.0   144843000.0  25450000.0   

        ebitda  totalAssets  totalCurrentLiabilities  totalShareholderEquity  \
78  29932000.0  663264000.0               63276000.0             588794000.0   
79  -6822000.0  686380000.0               59929000.0             612940000.0   
80  27305000.0  712390000.0               95806000.0             603046000.0   

    commonStockSharesOutstanding  operatingCashflow  
78                    1